In [1]:
# import necessary packages and helper function file
import math
import os
import itertools
import random
from ecc import *

In [2]:
"""
==================================================
    Define P-384 Curve
    from https://neuromancer.sk/std/nist/P-384
==================================================
"""
# big prime
p = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffeffffffff0000000000000000ffffffff

# define field over which we will define elliptic curve
K = GF(p)

# coefficients of curve equation y^2 = x^3 + ax + b
a = K(0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffeffffffff0000000000000000fffffffc)
b = K(0xb3312fa7e23ee7e4988e056be3f82d19181d9c6efe8141120314088f5013875ac656398d8a2ed19d2a85c8edd3ec2aef)

# build curve
E = EllipticCurve(K, (a, b))

# define base points
G = E(0xaa87ca22be8b05378eb1c71ef320ad746e1d3b628ba79b9859f741e082542a385502f25dbf55296c3a545e3872760ab7, 0x3617de4a96262c6f5d9e98bf9292dc29f8f41dbd289a147ce9da3113b5f0b8c00a60b1ce1d7e819d7a431d7c90ea0e5f)

# set order of curve (do this since order is already known...)
E.set_order(0xffffffffffffffffffffffffffffffffffffffffffffffffc7634d81f4372ddf581a0db248b0a77aecec196accc52973 * 0x1)

print(E.order)

<bound method EllipticCurve_finite_field.cardinality of Elliptic Curve defined by y^2 = x^3 + 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112316*x + 27580193559959705877849011840389048093056905856361568521428707301988689241309860865136260764883745107765439761230575 over Finite Field of size 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112319>


In [3]:
"""
==================================================
    Define Scheme Parameters
==================================================
"""
# sender, recipient private keys
d_s = random.randint(1, p)
d_r = random.randint(1,p)

# sender, recipient public keys
PU_s = d_s * G
PU_r = d_r * G

# initialize shared session key
k1 = PU_s * d_r
k2 = PU_r * d_s


IV = rand_bin(192)


In [4]:
"""
==================================================
    Sage Helper Functions 
==================================================
"""
def block_to_curve(block, E):
    """
    map individual block - an integer - to curve
    """
    x_coord = Integer(block)
    is_point = E.is_x_coord(x_coord)
    
    while(is_point == False):
        x_coord += 1
        is_point = E.is_x_coord(x_coord)
        
    foo = E.lift_x(x_coord)
    
    return foo


def msg_to_curve(message, E):
    """
    map entire message to curve using block_to_curve()
    """
    msg_points = []
    for block in message:
        msg_points.append(block_to_curve(block, E))
    return msg_points

def encrypt_msg(message, ksh):
    """
    encrypt message using shared session key (secret)
    """
    encrypted_msg = message
    for i in range(len(encrypted_msg)):
        encrypted_msg[i] = encrypted_msg[i] + ksh
    return encrypted_msg





In [5]:
"""
==================================================
    Read, Store, & Print Secret Message
==================================================
"""
file = open('secret_message.txt', 'r')
secret_message = file.read()
file.close()

print(secret_message)





I sat under the shade of the tree below Sweeney's Special after soloing Easy To Remember, and I just sat there in a daze thinking about the people in my life that I have lost. I just looked up at the steep, beautiful, colorful arête that shot straight up in the sky. It was like looking at a pathway to heaven. I was sitting in the great hall of Odin (a great legendary god). I felt the presence of late, great climbing partner Jimmy Forester and my late nephew who grew as a climber and person before being taken from us. Both of them fought the great battle of life. As I sat there, I could see holds to start appearing. I asked a good friend Matt King to help me put this route up. I wanted to put it up ground up, but it was so steep and nothing to hook. So I rapped in only to realize that after a short rappel I was out in space. So I heel hooked and basically downclimbed/ rappelled in to install each bolt. After ab crushing and bicep pumping work, we had all the bolts in except 1 at the sta

In [6]:
"""
==================================================
    Encode Secret Message
==================================================
"""
encodedMsg = encode_message(secret_message, IV)

In [9]:
"""
==================================================
    Map Secret Message to Curve, Encrypt, & Print
==================================================
"""
curve_msg = msg_to_curve(encodedMsg, E)

encrypted = encrypt_msg(curve_msg, k1)

writeable = list(encrypted)

print(writeable)

[(8646871157841558332207024854100485628562623815200003857127607533915665534806829402145161162647731187940965103070898 : 14563807195713254247386788581080790448100550611909679161841634687919788332115922323390498590491886074563336206422318 : 1), (35109563943237196459384181193697410281914529211963645837424978161201542899084279037757844938949631381218935171567467 : 3289967903833480669797389818649593616488298841108075340587477793058900165572569796047663783840178426723117868164614 : 1), (3141884060366987763829908810207032069832874649833729631795847175521423442453654440120372494079129974621365196555807 : 18146178438269896798476149969709666858470140640061780836397897497187247509412086048344936717045727795640319209111903 : 1), (25827755878725258167090110025923472353485349247428274981957416599288830240573639588196628730525763327804169527191251 : 36330373339657881767168475248284636239648298443510567274656014564079429357528960422802203315962733198936966520570783 : 1), (85184021812610029143867323011

In [8]:
"""
==================================================
Write secret message to .txt file
==================================================
"""

with open("encrypted_message.txt", 'w') as f:
    for point in writeable:
        f.write(str(point) + '\n')
    f.close
   
